In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


from sklearn.model_selection import train_test_split #splitting data
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer # scoring
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
%matplotlib inline
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)


In [ ]:

#Clean Data
raw = pd.read_csv('train.csv')

#Unnecessary features
del raw['Cabin']
del raw['Ticket']
del raw['Name']
del raw['PassengerId']

raw = raw.dropna(how = 'any')



#on-hot enconde all features that are categorical


raw['Parch_0'] = raw['Parch'].replace([0,1,2],[1,0,0])
raw['Parch_1'] = raw['Parch'].replace([0,1,2],[0,1,0])
raw['Parch_2'] = raw['Parch'].replace([0,1,2],[0,0,1])

raw['Pclass_1'] = raw['Pclass'].replace([1,2,3],[1,0,0])
raw['Pclass_2'] = raw['Pclass'].replace([1,2,3],[0,1,0])
raw['Pclass_3'] = raw['Pclass'].replace([1,2,3],[0,0,1])

raw['SibSp_0'] = raw['SibSp'].replace([0,1,2,3,4,5],[1,0,0,0,0,0])
raw['SibSp_1'] = raw['SibSp'].replace([0,1,2,3,4,5],[0,1,0,0,0,0])
raw['SibSp_2'] = raw['SibSp'].replace([0,1,2,3,4,5],[0,0,1,0,0,0])
raw['SibSp_3'] = raw['SibSp'].replace([0,1,2,3,4,5],[0,0,0,1,0,0])
raw['SibSp_4'] = raw['SibSp'].replace([0,1,2,3,4,5],[0,0,0,0,1,0])
raw['SibSp_5'] = raw['SibSp'].replace([0,1,2,3,4,5],[0,0,0,0,0,1])
del raw['SibSp']

del raw['Parch']
del raw['Pclass']


#Set up inputs and outputs
X = raw.drop('Survived',axis=1).copy()
X_encoded = pd.get_dummies(X, columns=['Embarked','Sex'])
y = raw['Survived'].copy()

raw.head()


In [ ]:
sum(y)/len(y)

In [ ]:
#BUILD LE TREE
X_train, X_test, y_train, y_test = train_test_split(X_encoded,y, random_state = 42, stratify = y)


In [ ]:
#Check to see if both the train set and test set have the same percentage of people who survived
(sum(y_test)/len(y_test)) == (sum(y_train)/len(y_train))

In [81]:
#BUILD LE TREE
clf_xgb= xgb.XGBClassifier(objective ='binary:logistic', seed = 42)
clf_xgb.fit(X_train,
            y_train,
            verbose = True,
            early_stopping_rounds=10,
            eval_metric ='aucpr',
            eval_set = [(X_test,y_test)])

[0]	validation_0-aucpr:0.79757
[1]	validation_0-aucpr:0.78881
[2]	validation_0-aucpr:0.78321
[3]	validation_0-aucpr:0.78367
[4]	validation_0-aucpr:0.77314
[5]	validation_0-aucpr:0.77521
[6]	validation_0-aucpr:0.77361
[7]	validation_0-aucpr:0.79388
[8]	validation_0-aucpr:0.79332
[9]	validation_0-aucpr:0.79736
[10]	validation_0-aucpr:0.80058
[11]	validation_0-aucpr:0.79799
[12]	validation_0-aucpr:0.80221
[13]	validation_0-aucpr:0.80676
[14]	validation_0-aucpr:0.80080
[15]	validation_0-aucpr:0.80345
[16]	validation_0-aucpr:0.80707
[17]	validation_0-aucpr:0.81276
[18]	validation_0-aucpr:0.81090
[19]	validation_0-aucpr:0.81125
[20]	validation_0-aucpr:0.81279
[21]	validation_0-aucpr:0.81212
[22]	validation_0-aucpr:0.81412
[23]	validation_0-aucpr:0.81500
[24]	validation_0-aucpr:0.81661
[25]	validation_0-aucpr:0.81844
[26]	validation_0-aucpr:0.81910
[27]	validation_0-aucpr:0.82247
[28]	validation_0-aucpr:0.82291
[29]	validation_0-aucpr:0.82311
[30]	validation_0-aucpr:0.82598
[31]	validation_0-

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=10,
              num_parallel_tree=1, predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [ ]:
plot_confusion_matrix(clf_xgb,
                      X_test,
                      y_test,
                      values_format = 'd')

In [82]:
#Optimize:

##Step 1
param_grid1 = {
    'max_depth':[4,5,6,7],
    'learning_rate':[0.01,0.1,0.3,0.5],
    'gamma':[0,0.25,1],
    'reg_lambda':[0,1,5,10],
    'scale_pos_weight':[1,3,5]
}

param_grid2 = {
    'max_depth':[4,6],
    'learning_rate':[0.3,0.5,0.6],
    'gamma':[0.1,0.25,0.3],
    'reg_lambda':[5,10,20,100],
    'scale_pos_weight':[5]
}


optimal_params = GridSearchCV(
    estimator = xgb.XGBClassifier(objective= 'binary:logistic',
                                  seed= 42,
                                  subsample= 0.9,
                                  colsample_bytree= 0.5),
    param_grid = param_grid2,
    scoring= 'roc_auc',
    verbose = 0,
    n_jobs = 10,
    cv = 3
)

optimal_params.fit(
        X_train,
            y_train,
            verbose = True,
            early_stopping_rounds=10,
            eval_metric ='aucpr',
            eval_set = [(X_test,y_test)]
)

print(optimal_params.best_params_)

/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[0]	validation_0-aucpr:0.69472
[1]	validation_0-aucpr:0.72253
[2]	validation_0-aucpr:0.78484
[3]	validation_0-aucpr:0.79073
[4]	validation_0-aucpr:0.80046
[5]	validation_0-aucpr:0.80899
[6]	validation_0-aucpr:0.80478
[7]	validation_0-aucpr:0.79690
[8]	validation_0-aucpr:0.79283
[9]	validation_0-aucpr:0.79512
[10]	validation_0-aucpr:0.76505


/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pan

[11]	validation_0-aucpr:0.76362
[12]	validation_0-aucpr:0.76605
[13]	validation_0-aucpr:0.76861
[14]	validation_0-aucpr:0.79397
[0]	validation_0-aucpr:0.70411
[1]	validation_0-aucpr:0.70544
[2]	validation_0-aucpr:0.79000
[3]	validation_0-aucpr:0.80011
[4]	validation_0-aucpr:0.81271
[5]	validation_0-aucpr:0.81995
[6]	validation_0-aucpr:0.82338
[7]	validation_0-aucpr:0.81581
[8]	validation_0-aucpr:0.81621
[9]	validation_0-aucpr:0.81368
[10]	validation_0-aucpr:0.81691
[11]	validation_0-aucpr:0.81298
[12]	validation_0-aucpr:0.81227
[13]	validation_0-aucpr:0.81442
[14]	validation_0-aucpr:0.80390
[15]	validation_0-aucpr:0.80543
[0]	validation_0-aucpr:0.68508
[1]	validation_0-aucpr:0.71027
[2]	validation_0-aucpr:0.79593
[3]	validation_0-aucpr:0.80187
[4]	validation_0-aucpr:0.80082
[5]	validation_0-aucpr:0.81409
[6]	validation_0-aucpr:0.81263
[7]	validation_0-aucpr:0.81651
[8]	validation_0-aucpr:0.81483
[9]	validation_0-aucpr:0.81761
[10]	validation_0-aucpr:0.81630
[11]	validation_0-aucpr:0.80

/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pan

...

[72]	validation_0-aucpr:0.81805
[18]	validation_0-aucpr:0.81731
[17]	validation_0-aucpr:0.81312
[18]	validation_0-aucpr:0.81466
[18]	validation_0-aucpr:0.80233
[16]	validation_0-aucpr:0.80605
[73]	validation_0-aucpr:0.81714
[19]	validation_0-aucpr:0.81072
[18]	validation_0-aucpr:0.81493
[17]	validation_0-aucpr:0.80371
[19]	validation_0-aucpr:0.80198
[74]	validation_0-aucpr:0.81818
[19]	validation_0-aucpr:0.80987
[20]	validation_0-aucpr:0.79906
[18]	validation_0-aucpr:0.80210
[75]	validation_0-aucpr:0.81818
[20]	validation_0-aucpr:0.81333
[21]	validation_0-aucpr:0.80210
[19]	validation_0-aucpr:0.80089
[21]	validation_0-aucpr:0.81583
[22]	validation_0-aucpr:0.81275
[20]	validation_0-aucpr:0.80238
[23]	validation_0-aucpr:0.81277
[22]	validation_0-aucpr:0.81851
[23]	validation_0-aucpr:0.81997
[24]	validation_0-aucpr:0.81790
[24]	validation_0-aucpr:0.82205
[25]	validation_0-aucpr:0.81778
[26]	validation_0-aucpr:0.81751
[25]	validation_0-aucpr:0.82112
[26]	validation_0-aucpr:0.82112
[27]	val

/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[31]	validation_0-aucpr:0.82211
[32]	validation_0-aucpr:0.81621
[33]	validation_0-aucpr:0.80875
[32]	validation_0-aucpr:0.82039
[34]	validation_0-aucpr:0.80964
[33]	validation_0-aucpr:0.81962
[35]	validation_0-aucpr:0.81231
[34]	validation_0-aucpr:0.81975
[35]	validation_0-aucpr:0.82209
[36]	validation_0-aucpr:0.81181
[37]	validation_0-aucpr:0.81170
[38]	validation_0-aucpr:0.81998
[36]	validation_0-aucpr:0.82259
[37]	validation_0-aucpr:0.82360
[39]	validation_0-aucpr:0.81915
[38]	validation_0-aucpr:0.82510
[40]	validation_0-aucpr:0.81915
[39]	validation_0-aucpr:0.82416
[40]	validation_0-aucpr:0.82415
[41]	validation_0-aucpr:0.82382
[42]	validation_0-aucpr:0.82422
[43]	validation_0-aucpr:0.82276
[44]	validation_0-aucpr:0.82321
[45]	validation_0-aucpr:0.82100
[46]	validation_0-aucpr:0.82157
[47]	validation_0-aucpr:0.82270
[48]	validation_0-aucpr:0.82220
[0]	validation_0-aucpr:0.70234
[1]	validation_0-aucpr:0.78564
[2]	validation_0-aucpr:0.77638
[3]	validation_0-aucpr:0.77533
[4]	validati

/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers st

[11]	validation_0-aucpr:0.81368
[12]	validation_0-aucpr:0.81476
[13]	validation_0-aucpr:0.81205
[14]	validation_0-aucpr:0.81449
[15]	validation_0-aucpr:0.81700
[16]	validation_0-aucpr:0.82282
[17]	validation_0-aucpr:0.81619
[18]	validation_0-aucpr:0.82356
[19]	validation_0-aucpr:0.82371
[20]	validation_0-aucpr:0.82625
[21]	validation_0-aucpr:0.82848
[22]	validation_0-aucpr:0.82586
[23]	validation_0-aucpr:0.82632
[24]	validation_0-aucpr:0.82597
[25]	validation_0-aucpr:0.82627
[26]	validation_0-aucpr:0.82600
[27]	validation_0-aucpr:0.82611
[28]	validation_0-aucpr:0.82561
[29]	validation_0-aucpr:0.82650
[30]	validation_0-aucpr:0.82701
[31]	validation_0-aucpr:0.82595
{'gamma': 0.1, 'learning_rate': 0.6, 'max_depth': 6, 'reg_lambda': 20, 'scale_pos_weight': 5}
